In [1]:
import os
import audio_mgmt
import deepspeech
import numpy as np
import pandas as pd
import wave
from pydub import AudioSegment
import mysql.connector
import json
import io

pygame 2.5.2 (SDL 2.28.3, Python 3.9.18)
Hello from the pygame community. https://www.pygame.org/contribute.html


C:\Users\thory\miniconda3\envs\Iota\lib\site-packages\pydub\utils.py:170: RuntimeWarning: Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work
  warn("Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work", RuntimeWarning)


In [2]:
# Load Pre-trained Model
ds = deepspeech.Model('Resources/deepspeech/pretrained/deepspeech-0.9.3-models.pbmm')

In [3]:
# Load constants
AUDIO_TRAINING_PATH = 'Resources/deepspeech/audio'

For training the DeepSpeech model with custom examples, we need to feed it audio files. Let us create some functions for reading and preprocessing audio files for training purposes, and storing them into database.

Let's start with creating a cursor for testing purposes.

In [4]:
conn = mysql.connector.connect(
    host='localhost',
    user='root',
    password='0301sonaL',
    database='chato_audio',
    auth_plugin='mysql_native_password'
)

cursor = conn.cursor()

**Read all transcripts into a list**

In [9]:
with open(os.path.join(AUDIO_TRAINING_PATH, 'transcripts.txt')) as fh:
    transcripts = fh.readlines()

**Store into database**

In [11]:
transcripts

['one two three four, get on the dance floor\n',
 'one two three four, get on the dance floor, yeah get on the dance floor\n',
 'Hey I want one cappuccino double double']

In [12]:
file = os.path.join(AUDIO_TRAINING_PATH, '003.wav')
audio_mgmt.store_into_database(file, transcripts[2], cursor)
conn.commit()

**Read from database**

In [20]:
row = audio_mgmt.read_wav_from_database(6, cursor)

In [21]:
row[0]

array([ 18770,  17990, -25692, ...,      7,      3,     -1], dtype=int16)

**Transcribe using DeepSpeech**

In [22]:
y_pred = ds.stt(row[0])
y = row[0]
print('Actual:\t', y)
print('Predicted:\t', y_pred)

Actual:	 [ 18770  17990 -25692 ...      7      3     -1]
Predicted:	 hey i won one capitine of the willavel


**Synthsize Text**

In [2]:
audio_mgmt.speak('That will be 4$ 45 cents, to the window please')